## DCF_1 ###

- Basic version of DCF 
- Acquire data from Yahoo finance using "yfinance"


step :
- get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash
- input > growth rate , terminal growth rate , discount rate , horizons
- calculate > FCF 
- calculate > terminal value
- calculate > present value
- calculate > Intrinsic value
- calculate > Margin of safety
- summary

In [80]:
import pandas as pd
import yfinance as yf

In [81]:
ticker = 'lh.bk'
data = yf.Ticker(ticker)
data.info

{'address1': 'Q House Lumpini Building',
 'address2': '37-38th Floors 1, South Satorn Road Thung Mahamek, Sathon',
 'city': 'Bangkok',
 'zip': '10120',
 'country': 'Thailand',
 'phone': '66 2 343 8900',
 'fax': '66 2 230 8130',
 'website': 'https://www.lh.co.th',
 'industry': 'Real Estate - Development',
 'industryKey': 'real-estate-development',
 'industryDisp': 'Real Estate - Development',
 'sector': 'Real Estate',
 'sectorKey': 'real-estate',
 'sectorDisp': 'Real Estate',
 'longBusinessSummary': "Land and Houses Public Company Limited engages in the property development activities in Thailand and the United States. It operates in two segments, Real Estate Business, and Rental and Service Business. The company's Real Estate Business segment develops and sells single detached houses, townhouses, and residence condominium projects. Its Rental and Service Business segment is involved in the rental of shopping malls, hotels, and apartments. The company also offers project administration 

In [82]:
### - get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash ###

## Shares ## 
shares = data.info['sharesOutstanding']
print('Ticker = ',ticker,' Shares = ',shares)

## Net Debt = Long Term Borrowings - Cash ###
totalnoncurrentliabilities = data.balancesheet.loc[data.balancesheet.index == 'Total Non Current Liabilities Net Minority Interest'].values[0][0] - data.balancesheet.loc[data.balancesheet.index == 'Cash And Cash Equivalents'].values[0][0]
print('Ticker = ',ticker,' Net Debt = ',totalnoncurrentliabilities)

## Average Free Cash Flow ## 
averagefreecashflow = data.cash_flow.T['Free Cash Flow'].dropna().mean()
print(data.cash_flow.T['Free Cash Flow'].dropna())
print('Ticker = ',ticker,'Average Free Cash Flow = ',averagefreecashflow)

Ticker =  lh.bk  Shares =  11949700096
Ticker =  lh.bk  Net Debt =  45720087529.0
2023-12-31   -6355455017.0
2022-12-31    5080856407.0
2021-12-31    6452482634.0
2020-12-31    2037881134.0
Name: Free Cash Flow, dtype: object
Ticker =  lh.bk Average Free Cash Flow =  1803941289.5


In [83]:
## - input > growth rate , terminal growth rate , discount rate , horizons ##
FCF = data.cash_flow.T['Operating Cash Flow'].dropna()
FCF = pd.DataFrame(FCF)
FCF = FCF.sort_index(ascending=True)
FCF['diff'] = FCF['Operating Cash Flow'].diff(1)
FCF['growth'] = FCF['diff']/FCF['Operating Cash Flow']
growthrate = FCF['growth'].mean()
print("Average Growth Rate from Operating Cash Flow = ",growthrate)

year_growth = 5
year_sustain = 5
discountrate = 0.10
terminalgrowthrate = 0.03
growthrate = 0.10

Average Growth Rate from Operating Cash Flow =  1.214366626786247


In [84]:
 ## - calculate > FCF ##

FCF_year_growth = [averagefreecashflow * ((1+growthrate)**i) for i in range(1,year_growth+1,1)]
print("FCF year growth",FCF_year_growth)

FCF_year_sustain = [FCF_year_growth[-1] * ((1+(growthrate/2))**i) for i in range(1,year_sustain+1,1)]
print("FCF year sustain",FCF_year_sustain)

## calculate > terminal value
terminalvalue = FCF_year_sustain[-1] * (1+(growthrate/2)) / (discountrate-terminalgrowthrate)
print("Terminal value = ",terminalvalue)


FCF year growth [np.float64(1984335418.45), np.float64(2182768960.2950006), np.float64(2401045856.3245006), np.float64(2641150441.9569507), np.float64(2905265486.152646)]
FCF year sustain [np.float64(3050528760.4602785), np.float64(3203055198.4832926), np.float64(3363207958.4074574), np.float64(3531368356.3278303), np.float64(3707936774.1442223)]
Terminal value =  55619051612.16334


In [85]:
## - calculate > present value ##
FCF_year_growth = FCF_year_growth + (FCF_year_sustain)
FCF_year_growth.append(terminalvalue)
pv = pd.DataFrame(FCF_year_growth,columns=['FCF'])
pv['pv'] = [pv['FCF'][i]/((1+discountrate)**(i+1)) for i in range(len(pv))]
print(pv)

totalpv = pv['pv'].sum() - totalnoncurrentliabilities
print('total pv = ',totalpv)

intrinsicvalue = totalpv/shares
print('intrinsicvalue = ',intrinsicvalue)

             FCF            pv
0   1.984335e+09  1.803941e+09
1   2.182769e+09  1.803941e+09
2   2.401046e+09  1.803941e+09
3   2.641150e+09  1.803941e+09
4   2.905265e+09  1.803941e+09
5   3.050529e+09  1.721944e+09
6   3.203055e+09  1.643674e+09
7   3.363208e+09  1.568961e+09
8   3.531368e+09  1.497645e+09
9   3.707937e+09  1.429570e+09
10  5.561905e+10  1.949414e+10
total pv =  -9344448675.902138
intrinsicvalue =  -0.7819818573547352
